In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/WhiteBoxAttacks/FGSM

/content/gdrive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master/WhiteBoxAttacks/FGSM


In [8]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/classifier

/content/gdrive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master/classifier


In [4]:
pip install bresenham  #line drawing algorithm

In [5]:
import torch
#from classifier.models import *
#from classifier.dataset import get_dataloader

import argparse

In [6]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master

/content/gdrive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master


In [9]:
from classifier.models import *
from classifier.dataset import get_dataloader

In [10]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/classifier

/content/gdrive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master/classifier


In [11]:
from classifier.models import *
from classifier.dataset import get_dataloader

In [12]:
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import torch.nn.functional as F #233
import torch.optim as optim
from torchvision import datasets,models,transforms
from PIL import Image
import PIL

In [13]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/WhiteBoxAttacks/FGSM

/content/gdrive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master/WhiteBoxAttacks/FGSM


In [14]:
from fgsm import FGSM

In [15]:
import sys
sys.argv=['']
del sys

In [16]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/

/content/gdrive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master


In [17]:
pwd

'/content/gdrive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master'

In [ ]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser(description='Skecth_Classification')
    parser.add_argument('--backbone_name', type=str, default='Resnet', help='VGG / InceptionV3/ Resnet')
    parser.add_argument('--pool_method', type=str, default='AdaptiveAvgPool2d', help='AdaptiveMaxPool2d / AdaptiveAvgPool2d / AvgPool2d')
    parser.add_argument('--batchsize', type=int, default=64)
    parser.add_argument('--nThreads', type=int, default=8)
    parser.add_argument('--splitTrain', type=float, default=0.8)
    parser.add_argument('--learning_rate', type=float, default=0.0001)

    hp = parser.parse_args()
    dataloader_Train, dataloader_Test = get_dataloader(hp)
    print(hp)

    hp.FGSM_MNIST = {'epsilon': 0.2, 'order': np.inf, 'clip_max': None, 'clip_min': None}

    model = Sketch_Classification(hp)
    model.to(device)
    model.load_state_dict(torch.load('./classifier/model_best_TUBerlin_Bina.pth', map_location=device))
    model.eval()

    with torch.no_grad():
        True_Accuracy = model.evaluate(dataloader_Test)

    print('True_Accuracy: {:.4f}, Adv_Accuracy: {}, Adv_Accuracy_preserved: {},  Time_Takes: {}\n'.format(True_Accuracy))

Total Training Sample 15999
Total Testing Sample 4000
Total Training Sample 15999
Total Testing Sample 4000
Namespace(backbone_name='Resnet', batchsize=64, learning_rate=0.0001, nThreads=8, pool_method='AdaptiveAvgPool2d', splitTrain=0.8)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser(description='Skecth_Classification')
    parser.add_argument('--backbone_name', type=str, default='Resnet', help='VGG / InceptionV3/ Resnet')
    parser.add_argument('--pool_method', type=str, default='AdaptiveAvgPool2d', help='AdaptiveMaxPool2d / AdaptiveAvgPool2d / AvgPool2d')
    parser.add_argument('--batchsize', type=int, default=64)
    parser.add_argument('--nThreads', type=int, default=8)
    parser.add_argument('--splitTrain', type=float, default=0.8)
    parser.add_argument('--learning_rate', type=float, default=0.0001)

    hp = parser.parse_args()
    dataloader_Train, dataloader_Test = get_dataloader(hp)
    print(hp)

    hp.FGSM_MNIST = {'epsilon': 0.2, 'order': np.inf, 'clip_max': None, 'clip_min': None}

    model = Sketch_Classification(hp)
    model.to(device)
    model.load_state_dict(torch.load('./classifier/model_best_TUBerlin_Bina.pth', map_location=device))
    model.eval()

    with torch.no_grad():
        True_Accuracy = 93.2250

    attack = FGSM(model, device='cuda')

    correct, correct_adv, correct_preserved = 0, 0, 0 #accuracy after noise injection
    test_loss, test_loss_adv = 0, 0
    start_time = time.time()

    for i_batch, batch in enumerate(dataloader_Test):

        print(i_batch)
        images = batch['sketch_img'].to(device)
        images = (images > 0.4).float()
        count=0
        for img, label in zip(images, batch['sketch_label']):
            target_label = torch.randint(0, 249, (1,))[0].to(device)
            img = img.unsqueeze(0)
            AdvExArray = attack.generate(img, target_label, **hp.FGSM_MNIST)
            # AdvExArray = AdvExArray.unsqueeze_(0).float()
            #images.save("geeks.jpg")
            output = model(AdvExArray)
            test_loss_adv += model.loss(output, label.to(device).unsqueeze(0)).item()
            prediction_adv = output.argmax(dim=1, keepdim=True).to('cpu')
            correct_adv += prediction_adv.eq(label.view_as(prediction_adv)).sum().item()

            AdvExArray_Bina =  (AdvExArray > 0.4).float()
            save_image(torch.cat((img, AdvExArray, AdvExArray_Bina), dim=0), '/content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/WhiteBoxAttacks/FGSM/Examples/images'+str(count)+'.jpg')
            # print(torch.equal(AdvExArray_Bina, img))
            output = model(AdvExArray_Bina)
            test_loss += model.loss(output, label.to(device).unsqueeze(0)).item()
            prediction = output.argmax(dim=1, keepdim=True).to('cpu')
            correct_preserved += prediction.eq(label.view_as(prediction)).sum().item()
            count+=1
    #        Save images
        break

    Adv_Accuracy = 100. * correct_adv / len(dataloader_Test.dataset)
    Adv_Accuracy_preserved = 100. * correct_preserved / len(dataloader_Test.dataset)
    print('True_Accuracy: {:.4f}, Adv_Accuracy: {}, Adv_Accuracy_preserved: {},  Time_Takes: {}\n'.format(True_Accuracy,
                                                                                                      Adv_Accuracy,
                                                                                                      Adv_Accuracy_preserved,
                                                                                                      (time.time() - start_time)))

Total Training Sample 15999
Total Testing Sample 4000
Total Training Sample 15999
Total Testing Sample 4000
Namespace(backbone_name='Resnet', batchsize=64, learning_rate=0.0001, nThreads=8, pool_method='AdaptiveAvgPool2d', splitTrain=0.8)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0
True_Accuracy: 93.2250, Adv_Accuracy: 0.0, Adv_Accuracy_preserved: 1.525,  Time_Takes: 18.204289436340332

